# Lightgbm两种形式的简单训练

### 原生形式使用lighgbm(import lightgbm as lgb)

In [3]:
# 从sklearn调入所需要的包
from sklearn import datasets
from sklearn.model_selection import train_test_split# 数据分割出训练集与验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score,recall_score
#导入鸢尾花数据
iris=datasets.load_iris()
data=iris.data
label=iris.target
print(pd.DataFrame(data).head())
print(pd.DataFrame(label).head())
data1=pd.DataFrame(data)
data1.columns=['sepal_l','sepal_w','petal_l','petal_w']
print(data1.head())
label1=pd.DataFrame(label)
label1.columns=['label']
print(label1.head())


     0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
   0
0  0
1  0
2  0
3  0
4  0
   sepal_l  sepal_w  petal_l  petal_w
0      5.1      3.5      1.4      0.2
1      4.9      3.0      1.4      0.2
2      4.7      3.2      1.3      0.2
3      4.6      3.1      1.5      0.2
4      5.0      3.6      1.4      0.2
   label
0      0
1      0
2      0
3      0
4      0


In [2]:
!pip install lightgbm 

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
# 划分训练集与测试集
train_x,test_x,train_y,test_y=train_test_split(data1,label1,test_size=0.3,random_state=42)
print('训练集长度：',len(train_x))
print('测试集长度:',len(test_x))

训练集长度： 105
测试集长度: 45


In [8]:
# 装换为DMatrix的数据格式
train_data=lgb.Dataset(train_x,train_y)
test_data=lgb.Dataset(test_x,test_y)
#设置参数
lgb_params={
    'boosting_type':'gbdt',
    'objective':'multiclass',
    'metric':'multi_error',
    'verboss':1,#<0显示致命的，=0 显示错误，>0显示信息
    'num_class':3 # lightgbm.basicLightGBMError:b"Number of classes should be specified and greater than 1 for multclass training"
}
# 模型训练
clf=lgb.train(lgb_params,train_data,num_boost_round=10,valid_sets=[train_data,test_data],verbose_eval=1)
#模型预测
test_pre=clf.predict(test_x,num_iteration=clf.best_iteration)
# 选择表示最高概率的列
test_pre_1=np.asarray([np.argmax(row) for row in test_pre])
print('test的预测结果:',test_pre_1)

[1]	training's multi_error: 0.133333	valid_1's multi_error: 0.155556
[2]	training's multi_error: 0.0571429	valid_1's multi_error: 0.0444444
[3]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[4]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[5]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[6]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[7]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[8]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[9]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[10]	training's multi_error: 0.0666667	valid_1's multi_error: 0
test的预测结果: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]


### sklearn 接口形式使用lightgbm

In [12]:
import lightgbm as lgb
lgb_params={
    'learing_rate':0.1,
    'max_bin':150,
    'num_leaves':32,
    'max_depth':11,
    'objective':'multiclass',
    'n_estimators':300    
}
model=lgb.LGBMClassifier(**lgb_params)
model.fit(train_x,train_y)
# 预测
# 输出预测值
test_pre2=model.predict(test_x)
print(test_pre2)
print('验证集精准率：',precision_score(test_y,test_pre2,average='macro'))
print('验证集召回率：',recall_score(test_y,test_pre2,average='macro'))

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
G:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
